# VQE Optimization Tracking with Qiskit

This notebook demonstrates how devqubit tracks a **VQE-style variational optimization**.

**What you'll see:**
- Step-by-step metric logging (energy convergence)
- Automatic circuit capture via backend wrapping
- Hyperparameter sweep with grouped runs
- Baseline setting and candidate verification
- Run comparison and portable bundling

**The problem:** We minimize the energy of a 1D Ising Hamiltonian using a hardware-efficient ansatz.

In [ ]:
from importlib.metadata import entry_points

if not any(ep.name == "qiskit" for ep in entry_points(group="devqubit.adapters")):
    raise ImportError(
        "devqubit Qiskit adapter is not installed.\n"
        "Install with: pip install 'devqubit[qiskit]'"
    )
else:
    print("Qiskit adapter available!")

In [ ]:
from pathlib import Path
import shutil
import numpy as np

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit_aer import AerSimulator

from devqubit import Config, set_config, track
from devqubit.compare import diff, verify_baseline
from devqubit.bundle import pack_run
from devqubit.compare import VerifyPolicy
from devqubit.runs import (
    set_baseline,
    load_run,
    list_runs,
    list_groups,
    list_runs_in_group,
)

### Setup

We configure devqubit to use a local demo workspace. In production, you'd typically:
- Set `DEVQUBIT_HOME` environment variable, or
- Use the default `~/.devqubit` directory

Here we use `Config(root_dir=...)` to isolate the demo data.

In [ ]:
# Problem configuration
PROJECT = "vqe_ising"
N_QUBITS = 4
N_LAYERS = 3
N_STEPS = 30
SHOTS = 2048
SEED = 42

# Exact ground energy for n-qubit Ising chain
EXACT_ENERGY = -(N_QUBITS - 1)

# Workspace setup
WORKSPACE = Path(".devqubit_vqe_demo")
if WORKSPACE.exists():
    shutil.rmtree(WORKSPACE)

set_config(Config(root_dir=WORKSPACE))

np.random.seed(SEED)
print(f"Workspace: {WORKSPACE.resolve()}")
print(f"Exact ground energy: {EXACT_ENERGY}")

### 1. Define the VQE Problem

**Hamiltonian:** $H = -\sum_{i=0}^{n-2} Z_i Z_{i+1}$ (1D Ising chain)

This Hamiltonian is diagonal in the computational basis, so we can estimate its energy directly from measurement counts — no basis rotations needed.

**Ansatz:** Hardware-efficient circuit with:
- RY rotation on each qubit per layer
- CNOT chain for entanglement
- Final measurement in computational basis

In [ ]:
def build_ansatz(n_qubits: int, n_layers: int):
    """Hardware-efficient ansatz with RY rotations and CNOT chain."""
    params = []
    qc = QuantumCircuit(n_qubits)

    for layer in range(n_layers):
        for q in range(n_qubits):
            p = Parameter(f"θ_{layer}_{q}")
            params.append(p)
            qc.ry(p, q)
        for q in range(n_qubits - 1):
            qc.cx(q, q + 1)

    qc.measure_all()
    return qc, params


def ising_energy(counts: dict, n_qubits: int) -> float:
    """Compute Ising energy ⟨H⟩ from measurement counts."""
    total = sum(counts.values())
    energy = 0.0

    for bitstring, count in counts.items():
        # Qiskit uses little-endian bit ordering
        bits = bitstring[::-1]
        spins = [1 if b == "0" else -1 for b in bits[:n_qubits]]
        e = sum(-spins[i] * spins[i + 1] for i in range(n_qubits - 1))
        energy += e * count

    return energy / total


def estimate_energy(backend, circuit, params, values, shots):
    """Bind parameters, run circuit, return energy estimate."""
    bound = circuit.assign_parameters(dict(zip(params, values)))
    job = backend.run(bound, shots=shots)
    counts = job.result().get_counts()
    return ising_energy(counts, circuit.num_qubits)

In [ ]:
ansatz, params = build_ansatz(N_QUBITS, N_LAYERS)

print(f"Ansatz: {len(params)} parameters, depth = {ansatz.depth()}")
print(ansatz.draw(fold=200))

### 2. VQE Optimizer

We use a simple **stochastic hill-climb** optimizer:
1. Start from random parameters
2. Propose a random perturbation
3. Accept if energy improves
4. Decay the step size over time

This is intentionally simple — real VQE would use gradient-based methods like SPSA or COBYLA.

In [ ]:
def run_vqe(backend, circuit, params, *, lr_init, lr_decay, n_steps, shots, seed, run):
    """Run VQE optimization, logging metrics at each step."""
    rng = np.random.default_rng(seed)

    theta = rng.uniform(-np.pi, np.pi, len(params))
    lr = lr_init

    best_energy = float("inf")
    best_theta = theta.copy()

    for step in range(n_steps):
        energy = estimate_energy(backend, circuit, params, theta, shots)

        if energy < best_energy:
            best_energy = energy
            best_theta = theta.copy()

        # Log step metrics to devqubit
        run.log_metric("energy", energy, step=step)
        run.log_metric("best_energy", best_energy, step=step)
        run.log_metric("lr", lr, step=step)

        # Propose perturbation and accept if better
        theta_trial = theta + rng.normal(0, lr, len(params))
        energy_trial = estimate_energy(backend, circuit, params, theta_trial, shots)

        if energy_trial < energy:
            theta = theta_trial

        lr *= lr_decay

    return best_theta, best_energy

### 3. Baseline Run

We run a single VQE optimization and track everything:
- **Parameters**: hyperparameters and problem config
- **Metrics**: energy at each step (time-series) + final summary
- **Artifacts**: best parameters found
- **Circuits**: automatically captured via `run.wrap(backend)`

The wrapped backend intercepts all circuit executions and stores them as artifacts.

In [ ]:
with track(project=PROJECT, run_name="baseline") as run:
    # Wrap backend - devqubit will capture all executed circuits
    backend = run.wrap(AerSimulator(seed_simulator=SEED))

    # Log configuration
    run.log_params(
        {
            "n_qubits": N_QUBITS,
            "n_layers": N_LAYERS,
            "n_steps": N_STEPS,
            "shots": SHOTS,
            "lr_init": 0.3,
            "lr_decay": 0.95,
            "optimizer": "stochastic_hillclimb",
        }
    )
    run.set_tags({"role": "baseline", "algorithm": "vqe", "hamiltonian": "ising_1d"})

    # Run optimization
    best_theta, best_energy = run_vqe(
        backend,
        ansatz,
        params,
        lr_init=0.3,
        lr_decay=0.95,
        n_steps=N_STEPS,
        shots=SHOTS,
        seed=SEED,
        run=run,
    )

    # Log summary metrics and best parameters
    run.log_metrics(
        {
            "final_energy": best_energy,
            "energy_error": best_energy - EXACT_ENERGY,
        }
    )
    run.log_json("best_params", {"theta": best_theta.tolist()}, role="result")

    baseline_id = run.run_id

print(f"Baseline run: {baseline_id}")
print(f"Best energy:  {best_energy:.4f}")
print(f"Exact energy: {EXACT_ENERGY}")
print(f"Error:        {best_energy - EXACT_ENERGY:.4f}")

### 4. Hyperparameter Sweep

Now let's compare different optimizer configurations. We use **grouped runs** to organize the sweep:
- All runs share the same `group_id`
- Easy to query and compare later
- Same random seed ensures fair comparison (same initial parameters)

We vary the learning rate and decay schedule to see which converges best.

In [ ]:
sweep_configs = [
    {"lr_init": 0.10, "lr_decay": 0.98, "name": "conservative"},
    {"lr_init": 0.30, "lr_decay": 0.95, "name": "moderate"},
    {"lr_init": 0.50, "lr_decay": 0.90, "name": "aggressive"},
]

sweep_results = []

print("Optimizer Sweep")
print("=" * 55)

for cfg in sweep_configs:
    with track(
        project=PROJECT,
        group_id="optimizer_sweep",
        group_name="Optimizer Comparison",
    ) as run:
        backend = run.wrap(AerSimulator(seed_simulator=SEED))

        run.log_params(
            {
                "n_qubits": N_QUBITS,
                "n_layers": N_LAYERS,
                "n_steps": N_STEPS,
                "shots": SHOTS,
                "lr_init": cfg["lr_init"],
                "lr_decay": cfg["lr_decay"],
                "optimizer": "stochastic_hillclimb",
            }
        )
        run.set_tags({"role": "sweep", "config": cfg["name"]})

        best_theta, best_energy = run_vqe(
            backend,
            ansatz,
            params,
            lr_init=cfg["lr_init"],
            lr_decay=cfg["lr_decay"],
            n_steps=N_STEPS,
            shots=SHOTS,
            seed=SEED,
            run=run,
        )

        run.log_metrics({"final_energy": best_energy})

        sweep_results.append(
            {
                "config": cfg["name"],
                "lr_init": cfg["lr_init"],
                "energy": best_energy,
                "run_id": run.run_id,
            }
        )

        print(
            f"  {cfg['name']:12s}  lr={cfg['lr_init']:.2f}  decay={cfg['lr_decay']:.2f}  E={best_energy:+.4f}"
        )

# Find winner (lowest energy)
winner = min(sweep_results, key=lambda r: r["energy"])
print(f"\n=> Winner: {winner['config']} (E = {winner['energy']:+.4f})")

### 5. Set Baseline and Verify Candidate

In a CI/CD workflow, you'd:
1. **Set a baseline** — the "known good" reference run
2. **Run candidates** — new code, different config, etc.
3. **Verify** — check that candidates meet quality criteria

devqubit's `verify_baseline()` checks:
- Parameter matching (optional)
- Program/circuit matching (optional)
- Distribution similarity via TVD (Total Variation Distance)

In [ ]:
# Set our baseline run as the project reference
set_baseline(PROJECT, baseline_id)
print(f"Baseline set: {baseline_id}")

Now run a **candidate** with the same configuration but a different simulator seed. This simulates what happens in production: same code, but different shot noise.

In [ ]:
with track(project=PROJECT, run_name="candidate") as run:
    # Different seed → different shot noise
    backend = run.wrap(AerSimulator(seed_simulator=SEED + 1))

    run.log_params(
        {
            "n_qubits": N_QUBITS,
            "n_layers": N_LAYERS,
            "n_steps": N_STEPS,
            "shots": SHOTS,
            "lr_init": 0.3,
            "lr_decay": 0.95,
            "optimizer": "stochastic_hillclimb",
        }
    )
    run.set_tags({"role": "candidate"})

    best_theta, best_energy = run_vqe(
        backend,
        ansatz,
        params,
        lr_init=0.3,
        lr_decay=0.95,
        n_steps=N_STEPS,
        shots=SHOTS,
        seed=SEED,
        run=run,
    )

    run.log_metrics({"final_energy": best_energy})
    candidate_id = run.run_id

print(f"Candidate run: {candidate_id}")
print(f"Energy: {best_energy:.4f}")

In [ ]:
# Verify candidate against baseline
policy = VerifyPolicy(
    params_must_match=True,  # Same configuration required
    program_must_match=True,  # Same circuit structure required
    tvd_max=0.15,  # Allow up to 15% distribution difference (shot noise)
)

result = verify_baseline(candidate_id, project=PROJECT, policy=policy)

print(f"Verification: {'[OK] PASSED' if result.ok else '[X] FAILED'}")
if not result.ok:
    print("Failures:")
    for f in result.failures:
        print(f"  - {f}")

### 6. Compare Runs

The `diff()` function produces a detailed comparison between two runs:
- Parameter differences
- Metric differences
- Program/circuit differences
- Distribution distance (TVD) when applicable

In [ ]:
comparison = diff(baseline_id, candidate_id)
print(comparison)

### 7. Query Runs and Groups

The registry stores all run metadata. You can:
- List all runs in a project
- List runs in a specific group (sweep)
- Load full run records for analysis

In [ ]:
print("All VQE Runs")
print("=" * 75)

for info in list_runs(project=PROJECT):
    rec = load_run(info["run_id"])
    energy = rec.metrics.get("final_energy", float("nan"))
    role = rec.tags.get("role", "-")
    config = rec.tags.get("config", rec.run_name or "-")
    quality = "✓" if energy < EXACT_ENERGY + 0.5 else ""

    print(f"  {rec.run_id[:12]}  {role:10s}  {config:12s}  E={energy:+.4f}  {quality}")

print(f"\nExact ground energy: {EXACT_ENERGY}")

In [ ]:
print("\nExperiment Groups")
print("-" * 40)

for g in list_groups():
    runs = list_runs_in_group(g["group_id"])
    print(f"  {g['group_name']}: {len(runs)} runs")

### 8. Bundle for Sharing

Bundles are self-contained ZIP files that include:
- Run metadata (params, metrics, tags)
- All artifacts (circuits, results, configs)
- Object store data (by content hash)

Share bundles with collaborators or attach to papers/PRs for reproducibility.

In [ ]:
bundle_path = WORKSPACE / "vqe_baseline.devqubit.zip"
result = pack_run(baseline_id, bundle_path)

print(f"Bundle created: {bundle_path.name}")
print(f"  Size:      {bundle_path.stat().st_size / 1024:.1f} KB")
print(f"  Artifacts: {result.artifact_count}")
print(f"  Objects:   {result.object_count}")

### Summary

In this notebook we demonstrated:

| Feature | What we used it for |
|---------|--------------------|
| `track()` | Create tracked runs with automatic finalization |
| `run.wrap(backend)` | Capture all circuit executions |
| `run.log_metric(..., step=)` | Time-series metrics (energy per step) |
| `group_id` | Organize hyperparameter sweep runs |
| `registry.set_baseline()` | Mark a run as the reference |
| `verify_baseline()` | CI/CD-style quality checks |
| `diff()` | Human-readable run comparison |
| `pack_run()` | Portable bundle for sharing |

In [ ]:
# Cleanup demo workspace
shutil.rmtree(WORKSPACE)
print("Done!")